# Updating the ETL script to incorporate weight forecast model

## Setup

In [1]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

%load_ext autoreload
%autoreload 2

from wtfc_utils import etl_utils as etl

from sqlalchemy import create_engine

import datetime

import numpy as np
np.set_printoptions(threshold=500)
import pandas as pd
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import altair as alt

## Load Data

In [49]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
db_dir = server_dir + 'db/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

engine = create_engine('sqlite:///'+db_file_name)

with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])

In [50]:
db_df.tail()

,weight,calories,steps,weight_imputed,weight_measured,w_7day_avg,c_7day_avg,s_7day_avg,w_7day_avg_last_week,c_7day_avg_last_week,s_7day_avg_last_week,w_7day_avg_weekly_diff,Mv1_0_weight_velocity,Mv1_0_proj_weight_1mo,Mv1_0_proj_weight_2mo,Mv1_0_target_steps,Mv1_0_target_cals,Mv1_0_targ_weight_1mo,Mv1_0_targ_weight_2mo
date,,,,,,,,,,,,,,,,,,,
2021-07-09,154.9,2006.0,17747.0,0.0,154.9,156.942857,2080.428571,17680.142857,157.857143,2263.714286,14845.000000,-0.914286,-0.411370,155.232929,153.635718,17680.142857,2035.694721,154.864526,152.923196
2021-07-10,158.6,2848.0,14936.0,0.0,158.6,157.014286,2121.714286,16423.285714,157.500000,2288.000000,16686.428571,-0.485714,-0.272153,155.883039,154.826363,16423.285714,2006.713366,154.935954,152.994625
2021-07-11,159.8,2352.0,10991.0,0.0,159.8,157.085714,2130.714286,15578.714286,157.842857,2313.142857,17260.714286,-0.757143,-0.216108,156.187424,155.348349,15578.714286,1987.426284,155.007383,153.066053
2021-07-12,157.2,2036.0,15940.0,0.0,157.2,157.042857,2120.428571,15494.857143,158.071429,2280.857143,16800.857143,-1.028571,-0.231901,156.078923,155.178531,15494.857143,1985.111469,154.964526,153.023196
2021-07-13,158.7,1650.0,1803.0,0.0,158.7,157.471429,2087.428571,13257.857143,157.800000,2288.285714,17242.142857,-0.328571,-0.199865,156.640656,155.864647,13257.857143,1935.942223,155.393097,153.451767


In [44]:
# Model v0.1
c_w = 0.9842664081035283
c_c = 0.001965638199353011
c_s = -4.621900527451458e-05
c_0 = -1.2110620297640367
alpha_s = -c_s/c_c
alpha_0 = -c_0/c_c
alpha_w = (1-c_w)/c_c
gamma = -np.log(c_w)
steps_goal = 10000
weight_velocity_goal = -0.5

def w_next_week(w_curr_week, c_next_week, s_next_week):
    return c_0 + c_w*w_curr_week + c_c*c_next_week + c_s*s_next_week

def w_steady_state(c, s):
    return (c - alpha_s*s - alpha_0)/alpha_w

def c_steady_state(w, s):
    return (alpha_s*s + alpha_0 + alpha_w*w)

def w_forecast(t_weeks, w_curr_week, c, s):
    wss = w_steady_state(c, s)
    return (w_curr_week - wss)*np.exp(-gamma*t_weeks) + wss

def w_velocity(w_curr_week, c, s):
    wss = w_steady_state(c, s)
    return gamma*(wss - w_curr_week)
    
def steps_target(s_goal, s_avg):
    return max(s_goal, s_avg)

def cals_target(w_vel, w_curr_week, s):
    return alpha_w*(w_vel/gamma + w_curr_week) + alpha_s*s + alpha_0

In [54]:
wgt = 156
cal = 2185.97
steps = 14000

print(w_next_week(wgt, cal, steps))
print(w_forecast(1, wgt, cal, steps))

print(w_steady_state(cal, steps))
print(c_steady_state(wgt, steps))

print(w_velocity(wgt, cal, steps))
dt = 1
print((w_forecast(dt, wgt, cal, steps)-wgt)/dt)

print(steps_target(steps_goal, steps))
print(cals_target(weight_velocity_goal, wgt, steps))

155.98425769518286
155.98425769518286
154.999446221771
2193.9787499430513
-0.015867460821684578
-0.01574230481713812
14000
1941.6148073736592
